In [1]:
import sys
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("loan.csv")

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def getClass(status):
    if status=="Charged Off" or status=="Default" or status=="Does not meet the credit policy. Status:Charged Off" or status=="In Grace Period" or status=="Late (16-30 days)" or status=="Late (31-120 days)":
        return 0
    else:
        return 1
df["goodLoan"] = df["loan_status"].apply(lambda x: getClass(x))

In [4]:
midWest = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
northEast = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']
southEast = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
southWest = ['AZ', 'TX', 'NM', 'OK']
df['region'] = np.nan
def getRegion(state):
    if state in west:
        return 'West'
    elif state in southWest:
        return 'SouthWest'
    elif state in southEast:
        return 'SouthEast'
    elif state in midWest:
        return 'MidWest'
    elif state in northEast:
        return 'NorthEast'
df['region'] = df['addr_state'].apply(lambda x: getRegion(x))

In [5]:
def getIncomeCat(income):
    if income<50000:
        return "Low"
    elif income>=50000 and income<100000:
        return "Middle"
    else: 
        return "High"
df["incomeCat"] = df["annual_inc"].apply(lambda x: getIncomeCat(x))

In [6]:
finaldf = df.copy()
"""
pub_rec - # of accounts client is delinquent
total_acc - # of credit lines the borrower has
delinq_2yrs - # of delinquencies in past 2 yrs
"""
finaldf = finaldf[['loan_amnt', "region", "incomeCat", 'term', 'int_rate', 'installment', 'grade',
       'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan',
       'purpose', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 
       'goodLoan', 'mths_since_last_major_derog', 'collection_recovery_fee', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
        'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
        'last_pymnt_amnt']]

In [7]:
df_subset = finaldf.sample(n=10000)

In [8]:
df_subset.select_dtypes(include=['object']).head()

,region,incomeCat,term,grade,home_ownership,verification_status,pymnt_plan,purpose
939182,SouthEast,Middle,36 months,E,RENT,Verified,n,debt_consolidation
180306,SouthEast,Middle,60 months,A,MORTGAGE,Source Verified,n,credit_card
1210670,MidWest,Middle,36 months,C,MORTGAGE,Not Verified,n,debt_consolidation
1319253,MidWest,High,36 months,C,MORTGAGE,Verified,n,debt_consolidation
1982352,West,High,60 months,D,MORTGAGE,Verified,n,debt_consolidation


In [9]:
col_object = df_subset.select_dtypes(include=['object']).columns.values.tolist()
col_object

['region',
 'incomeCat',
 'term',
 'grade',
 'home_ownership',
 'verification_status',
 'pymnt_plan',
 'purpose']

In [10]:
expDf = pd.get_dummies(data = df_subset, columns = col_object)

In [11]:
expDf.shape

(10000, 66)

In [12]:
expCol = expDf.columns.values
expCol

array(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'goodLoan',
       'mths_since_last_major_derog', 'collection_recovery_fee',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'last_pymnt_amnt', 'region_MidWest', 'region_NorthEast',
       'region_SouthEast', 'region_SouthWest', 'region_West',
       'incomeCat_High', 'incomeCat_Low', 'incomeCat_Middle',
       'term_ 36 months', 'term_ 60 months', 'grade_A', 'grade_B',
       'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G',
       'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified',
       'verificati

In [13]:
expDf = expDf.dropna(axis='columns')
testdf = expDf.copy()

In [14]:
expCol1 = ['loan_amnt', 'int_rate', 'installment', 'annual_inc',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'total_acc', 'collection_recovery_fee',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'last_pymnt_amnt', 'region_MidWest', 'region_NorthEast',
       'region_SouthEast', 'region_SouthWest', 'region_West',
       'incomeCat_High', 'incomeCat_Low', 'incomeCat_Middle',
       'term_ 36 months', 'term_ 60 months', 'grade_A', 'grade_B',
       'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G',
       'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified',
       'verification_status_Verified', 'pymnt_plan_n', 'pymnt_plan_y']
for i in range(len(expCol1)):
    for j in range(len(expCol1)):
        testdf['{}_{}'.format(expCol1[i], expCol1[j])] = testdf[expCol1[i]]*testdf[expCol1[j]]
testdf.head()

,loan_amnt,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,...,pymnt_plan_y_grade_G,pymnt_plan_y_home_ownership_ANY,pymnt_plan_y_home_ownership_MORTGAGE,pymnt_plan_y_home_ownership_OWN,pymnt_plan_y_home_ownership_RENT,pymnt_plan_y_verification_status_Not Verified,pymnt_plan_y_verification_status_Source Verified,pymnt_plan_y_verification_status_Verified,pymnt_plan_y_pymnt_plan_n,pymnt_plan_y_pymnt_plan_y
939182,10000,18.20,362.53,65000.0,0.0,3.0,23.0,0.0,11905,51.0,...,0,0,0,0,0,0,0,0,0,0
180306,12000,6.11,232.61,75000.0,0.0,0.0,10.0,0.0,10063,18.0,...,0,0,0,0,0,0,0,0,0,0
1210670,9000,12.29,300.18,73000.0,0.0,1.0,33.0,1.0,10265,49.0,...,0,0,0,0,0,0,0,0,0,0
1319253,7000,14.99,242.63,135000.0,0.0,0.0,8.0,0.0,10141,34.0,...,0,0,0,0,0,0,0,0,0,0
1982352,30000,16.29,734.18,112000.0,0.0,3.0,10.0,0.0,21810,30.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
expDf = testdf.copy()
expDf.shape

(10000, 2086)

In [16]:
expDf.columns.tolist()

['loan_amnt',
 'int_rate',
 'installment',
 'annual_inc',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'goodLoan',
 'collection_recovery_fee',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'last_pymnt_amnt',
 'region_MidWest',
 'region_NorthEast',
 'region_SouthEast',
 'region_SouthWest',
 'region_West',
 'incomeCat_High',
 'incomeCat_Low',
 'incomeCat_Middle',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'verification_status_Not Verified',
 'verification_status_Source Verified',
 'verification_status_Verified',
 'pymnt_plan_n',
 'pymnt_plan_y',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',

In [17]:
expDf=expDf.drop('goodLoan', axis=1)

In [18]:
purposeList = ['purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding']
purposeDict = dict(zip(range(14), purposeList))
len(purposeList)
purposeDict

{0: 'purpose_car',
 1: 'purpose_credit_card',
 2: 'purpose_debt_consolidation',
 3: 'purpose_educational',
 4: 'purpose_home_improvement',
 5: 'purpose_house',
 6: 'purpose_major_purchase',
 7: 'purpose_medical',
 8: 'purpose_moving',
 9: 'purpose_other',
 10: 'purpose_renewable_energy',
 11: 'purpose_small_business',
 12: 'purpose_vacation',
 13: 'purpose_wedding'}

In [19]:
dfPurpose = []
dfCar = expDf[expDf['purpose_car']==1].reset_index(drop=True)
dfCreditCard = expDf[expDf['purpose_credit_card']==1].reset_index(drop=True)
dfDebtCon = expDf[expDf['purpose_debt_consolidation']==1].reset_index(drop=True)
dfEdu = expDf[expDf['purpose_educational']==1].reset_index(drop=True)
dfHomeImprov = expDf[expDf['purpose_home_improvement']==1].reset_index(drop=True)
dfHouse = expDf[expDf['purpose_house']==1].reset_index(drop=True)
dfMajorPurch = expDf[expDf['purpose_major_purchase']==1].reset_index(drop=True)
dfMed = expDf[expDf['purpose_medical']==1].reset_index(drop=True)
dfMoving = expDf[expDf['purpose_moving']==1].reset_index(drop=True)
dfOther = expDf[expDf['purpose_other']==1].reset_index(drop=True)
dfRenew = expDf[expDf['purpose_renewable_energy']==1].reset_index(drop=True)
dfSmall = expDf[expDf['purpose_small_business']==1].reset_index(drop=True)
dfVaca = expDf[expDf['purpose_vacation']==1].reset_index(drop=True)
dfWed = expDf[expDf['purpose_wedding']==1].reset_index(drop=True)
dfPurpose = [dfCar, dfCreditCard, dfDebtCon, dfEdu, dfHomeImprov, dfHouse, dfMajorPurch, dfMed, dfMoving, 
             dfOther, dfRenew, dfSmall, dfVaca, dfWed]
dfw = [dfCar.copy(), dfCreditCard.copy(), dfDebtCon.copy(), dfEdu.copy(), dfHomeImprov.copy(), dfHouse.copy(), 
       dfMajorPurch.copy(), dfMed.copy(), dfMoving.copy(), dfOther.copy(), dfRenew.copy(), dfSmall.copy(), 
       dfVaca.copy(), dfWed.copy()]

In [26]:
expDf.shape

(10000, 2085)

In [27]:
dfwCar = dfw[0].as_matrix()
qList = []
for row in dfwCar:
    q = np.linalg.norm(row, ord=2)
    qList.append(q)
qList = [q/sum(qList) for q in qList]
x = np.random.choice(range(dfwCar.shape[0]), 1, p=qList)[0]
# S = np.append(S,[dfwCar[x]], axis=0)
x_tilda = dfwCar[x]
dfwCar[0] = dfwCar[0] - (np.dot(dfwCar[0],x_tilda))/ ((np.linalg.norm(x_tilda, ord=2))**2)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [28]:
# dfw.as_matrix()[0] -= np.dot(dfw.as_matrix()[0], x_tilda)/((np.linalg.norm(x_tilda, ord=2))**2)

In [29]:
dfw = expDf.reset_index(drop=True)
x_tilda
dfw.loc[0] - np.dot(dfw.loc[0], x_tilda)/(np.linalg.norm(x_tilda, ord=2)**2)
# for i in range(dfw.shape[0]):
# dfw.loc[0] -= np.dot(dfw.loc[0], x_tilda)/((np.linalg.norm(x_tilda, ord=2))**2)

loan_amnt                                            9999.465521
int_rate                                               17.665521
installment                                           361.995521
annual_inc                                          64999.465521
delinq_2yrs                                            -0.534479
inq_last_6mths                                          2.465521
open_acc                                               22.465521
pub_rec                                                -0.534479
revol_bal                                           11904.465521
total_acc                                              50.465521
collection_recovery_fee                               354.555121
out_prncp                                              -0.534479
out_prncp_inv                                          -0.534479
total_pymnt                                          4131.315521
total_pymnt_inv                                      4069.335521
total_rec_int            

In [30]:
k = 1400
S = np.empty([0, expDf.shape[1]])
dfS = pd.DataFrame(columns = expDf.columns.values.tolist())
kList = [int(k/14)]*14
dfw = expDf.reset_index(drop=True)
X = dfw.as_matrix()
dfPurpose = [dfCar, dfCreditCard, dfDebtCon, dfEdu, dfHomeImprov, dfHouse, dfMajorPurch, dfMed, dfMoving, 
             dfOther, dfRenew, dfSmall, dfVaca, dfWed]
dfw = [dfCar.copy(), dfCreditCard.copy(), dfDebtCon.copy(), dfEdu.copy(), dfHomeImprov.copy(), dfHouse.copy(), 
       dfMajorPurch.copy(), dfMed.copy(), dfMoving.copy(), dfOther.copy(), dfRenew.copy(), dfSmall.copy(), 
       dfVaca.copy(), dfWed.copy()]
countCar, countCreditCard, countDebtCon, countEdu, countHomeImprov, countHouse, countMajorPurch = 0, 0, 0, 0, 0, 0, 0
countMed, countMoving, countOther, countRenew, countSmall, countVaca, countWed = 0, 0, 0, 0, 0, 0, 0
countList = [countCar, countCreditCard, countDebtCon, countEdu, countHomeImprov, countHouse, countMajorPurch, 
            countMed, countMoving, countOther, countRenew, countSmall, countVaca, countWed]
len(dfS)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


0

In [32]:
t = 0
dfS = pd.DataFrame(columns = expDf.columns.values.tolist())
while dfS.shape[0] < k:
    t += 1
    print("interation (t): {}".format(t))
    i = np.random.choice(14,1)[0]
    X_chosen = dfPurpose[i]
    X_name = purposeDict[i]
    print("Chosen class is: {}".format(X_name))
    X_i = X_chosen.shape[0]
    X_w_chosen = dfw[i].as_matrix()
    print(X_w_chosen)
    for name in purposeList:
        if name == X_name:
            countList[i] += 1
    if countList[i] < kList[0]:
        qList = []
        for row in X_w_chosen:
            q = np.linalg.norm(row, ord=2)**2
            qList.append(q)
        qList = [q/sum(qList) for q in qList]
        x_i = np.random.choice(range(X_i), 1, p=qList)[0]
        print("Chosen random sample is No.{}".format(x_i))
        x_tilda = X_chosen.as_matrix()[x_i]
        dfS.loc[len(dfS)] = x_tilda   
        for p in range(14):
            X_w_update = dfw[p].as_matrix()
            for i in range(X_w_update.shape[0]):
                X_w_update[i] -= np.dot(X_w_update[i], x_tilda)/((np.linalg.norm(x_tilda, ord=2))**2)
            dfw[p] = pd.DataFrame(X_w_update, columns = expDf.columns.values.tolist()).reset_index(drop=True)
#         X = dfw.as_matrix()
#         for i in range(X.shape[0]):
#             X[i] -= np.dot(X[i], x_tilda)/((np.linalg.norm(x_tilda, ord=2))**2)
#         dfw = pd.DataFrame(X, columns = expDf.columns.values.tolist()).reset_index(drop=True)
    else: break
    
print(dfS)

interation (t): 1
Chosen class is: purpose_moving
[[ 1.59993806e+04  2.66506021e+01  4.90540602e+02 ... -6.19397871e-01
  -6.19397871e-01 -6.19397871e-01]
 [ 1.79962151e+04  7.32507251e+00  5.86455073e+02 ... -3.78492749e+00
  -3.78492749e+00 -3.78492749e+00]
 [ 5.59869201e+03  1.46820149e+01  1.95552015e+02 ... -1.30798508e+00
  -1.30798508e+00 -1.30798508e+00]
 ...
 [ 1.19994294e+04  2.34194469e+01  3.44579447e+02 ... -5.70553123e-01
  -5.70553123e-01 -5.70553123e-01]
 [ 1.79982177e+04  9.52773165e+00  5.90167732e+02 ... -1.78226835e+00
  -1.78226835e+00 -1.78226835e+00]
 [ 9.99934241e+02  1.74042407e+01  3.58242407e+01 ... -6.57593047e-02
  -6.57593047e-02 -6.57593047e-02]]
Chosen random sample is No.54


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


interation (t): 2
Chosen class is: purpose_educational
[[ 1.99343884e+03  6.04883915e+00  6.04588391e+01 ... -6.56116085e+00
  -6.56116085e+00 -6.56116085e+00]
 [ 1.99982906e+03  1.58290649e+01  7.01490649e+01 ... -1.70935057e-01
  -1.70935057e-01 -1.70935057e-01]]
Chosen random sample is No.0
interation (t): 3
Chosen class is: purpose_medical
[[ 3.58593215e+03  1.54214990e+00  1.11812150e+02 ... -1.40678501e+01
  -1.40678501e+01 -1.40678501e+01]
 [ 5.99881896e+03  1.77589637e+01  2.18578964e+02 ... -1.18103634e+00
  -1.18103634e+00 -1.18103634e+00]
 [ 5.99911266e+03  1.80526575e+01  2.18872657e+02 ... -8.87342522e-01
  -8.87342522e-01 -8.87342522e-01]
 ...
 [ 2.49570424e+03  8.49423572e+00  7.96942357e+01 ... -4.29576428e+00
  -4.29576428e+00 -4.29576428e+00]
 [ 2.79682630e+03  1.50662998e+01  9.83962998e+01 ... -3.17370025e+00
  -3.17370025e+00 -3.17370025e+00]
 [ 8.39977040e+03  7.11039568e+00  2.60450396e+02 ... -2.29604320e-01
  -2.29604320e-01 -2.29604320e-01]]
Chosen random samp

interation (t): 15
Chosen class is: purpose_major_purchase
[[ 4.99754341e+03  1.84334096e+01  1.85643410e+02 ... -2.45659045e+00
  -2.45659045e+00 -2.45659045e+00]
 [ 5.99859817e+03  1.21881724e+01  2.02478172e+02 ... -1.40182760e+00
  -1.40182760e+00 -1.40182760e+00]
 [ 9.99597053e+03  5.73053245e+00  3.17520532e+02 ... -4.02946755e+00
  -4.02946755e+00 -4.02946755e+00]
 ...
 [ 1.19867582e+04  4.81817456e+00  4.20958175e+02 ... -1.32418254e+01
  -1.32418254e+01 -1.32418254e+01]
 [ 1.99794073e+03  1.19307349e+01  6.62907349e+01 ... -2.05926508e+00
  -2.05926508e+00 -2.05926508e+00]
 [ 1.04977748e+04  7.52484930e+00  3.35354849e+02 ... -2.22515070e+00
  -2.22515070e+00 -2.22515070e+00]]
Chosen random sample is No.35
interation (t): 16
Chosen class is: purpose_credit_card
[[ 1.19951853e+04  1.29529916e+00  2.27795299e+02 ... -4.81470084e+00
  -4.81470084e+00 -4.81470084e+00]
 [ 8.39858698e+03  1.15769849e+01  2.81576985e+02 ... -1.41301514e+00
  -1.41301514e+00 -1.41301514e+00]
 [ 9.9989

interation (t): 27
Chosen class is: purpose_moving
[[ 1.59969289e+04  2.41989419e+01  4.88088942e+02 ... -3.07105806e+00
  -3.07105806e+00 -3.07105806e+00]
 [ 1.79808873e+04 -8.00266475e+00  5.71127335e+02 ... -1.91126647e+01
  -1.91126647e+01 -1.91126647e+01]
 [ 5.59358158e+03  9.57158354e+00  1.90441584e+02 ... -6.41841646e+00
  -6.41841646e+00 -6.41841646e+00]
 ...
 [ 1.19969412e+04  2.09311769e+01  3.42091177e+02 ... -3.05882308e+00
  -3.05882308e+00 -3.05882308e+00]
 [ 1.79912865e+04  2.59647244e+00  5.83236472e+02 ... -8.71352756e+00
  -8.71352756e+00 -8.71352756e+00]
 [ 9.99680543e+02  1.71505433e+01  3.55705433e+01 ... -3.19456686e-01
  -3.19456686e-01 -3.19456686e-01]]
Chosen random sample is No.25
interation (t): 28
Chosen class is: purpose_educational
[[ 1.97184063e+03 -1.55493709e+01  3.88606291e+01 ... -2.81593709e+01
  -2.81593709e+01 -2.81593709e+01]
 [ 1.99924465e+03  1.52446491e+01  6.95646491e+01 ... -7.55350888e-01
  -7.55350888e-01 -7.55350888e-01]]
Chosen random sa

...

interation (t): 1181
Chosen class is: purpose_debt_consolidation
[[ 9.76910125e+03 -2.12698749e+02  1.31631251e+02 ... -2.30898749e+02
  -2.30898749e+02 -2.30898749e+02]
 [ 8.70248339e+03 -2.85226614e+02  2.66338555e+00 ... -2.97516614e+02
  -2.97516614e+02 -2.97516614e+02]
 [ 6.05631222e+03 -9.28697778e+02 -7.01057778e+02 ... -9.43687778e+02
  -9.43687778e+02 -9.43687778e+02]
 ...
 [ 1.39141925e+04 -5.69347469e+02 -2.29637469e+02 ... -5.85807469e+02
  -5.85807469e+02 -5.85807469e+02]
 [ 2.16839591e+04 -3.05050850e+02  4.04109150e+02 ... -3.16040850e+02
  -3.16040850e+02 -3.16040850e+02]
 [ 1.73908135e+04 -3.99436529e+02  1.63083471e+02 ... -4.09186529e+02
  -4.09186529e+02 -4.09186529e+02]]
Chosen random sample is No.239
interation (t): 1182
Chosen class is: purpose_wedding
[[ 3.81507127e+03 -1.77938731e+02 -6.14287309e+01 ... -1.84928731e+02
  -1.84928731e+02 -1.84928731e+02]
 [ 3.91864636e+03 -6.16336372e+01  6.67363628e+01 ... -8.13536372e+01
  -8.13536372e+01 -8.13536372e+01]
 [ 6

interation (t): 1193
Chosen class is: purpose_wedding
[[ 3.81362172e+03 -1.79388276e+02 -6.28782758e+01 ... -1.86378276e+02
  -1.86378276e+02 -1.86378276e+02]
 [ 3.91802546e+03 -6.22545406e+01  6.61154594e+01 ... -8.19745406e+01
  -8.19745406e+01 -8.19745406e+01]
 [ 6.54024406e+03 -4.50125937e+02 -2.35095937e+02 ... -4.59755937e+02
  -4.59755937e+02 -4.59755937e+02]
 ...
 [ 4.80353477e+03 -1.76745233e+02 -1.13552333e+01 ... -1.96465233e+02
  -1.96465233e+02 -1.96465233e+02]
 [ 1.80766134e+04 -1.24556626e+02  3.74113374e+02 ... -1.48386626e+02
  -1.48386626e+02 -1.48386626e+02]
 [ 3.33707984e+03 -5.27601597e+01  4.70498403e+01 ... -6.29201597e+01
  -6.29201597e+01 -6.29201597e+01]]
Chosen random sample is No.3
interation (t): 1194
Chosen class is: purpose_vacation
[[ 5.64593835e+03 -3.36071651e+02 -1.37171651e+02 ... -3.54061651e+02
  -3.54061651e+02 -3.54061651e+02]
 [ 1.84297424e+04 -7.58767598e+02 -1.37207598e+02 ... -7.70257598e+02
  -7.70257598e+02 -7.70257598e+02]
 [ 1.72419703e+0

interation (t): 1206
Chosen class is: purpose_renewable_energy
[[17785.62126659  -197.28873341   233.84126659 ...  -214.37873341
   -214.37873341  -214.37873341]
 [11245.97788798  -727.72211202  -392.60211202 ...  -754.02211202
   -754.02211202  -754.02211202]
 [ 2780.31614338  -205.69385662  -117.16385662 ...  -219.68385662
   -219.68385662  -219.68385662]
 ...
 [ 9610.72177996  -383.96822004   -88.16822004 ...  -389.27822004
   -389.27822004  -389.27822004]
 [ 8300.06583697   -80.94416303   207.93583697 ...   -99.93416303
    -99.93416303   -99.93416303]
 [ 3892.74411922   -39.26588078    85.53411922 ...   -57.25588078
    -57.25588078   -57.25588078]]
Chosen random sample is No.6
interation (t): 1207
Chosen class is: purpose_wedding
[[ 3.80971237e+03 -1.83297629e+02 -6.67876293e+01 ... -1.90287629e+02
  -1.90287629e+02 -1.90287629e+02]
 [ 3.91636796e+03 -6.39120363e+01  6.44579637e+01 ... -8.36320363e+01
  -8.36320363e+01 -8.36320363e+01]
 [ 6.53068875e+03 -4.59681250e+02 -2.4465125

interation (t): 1219
Chosen class is: purpose_credit_card
[[11685.88354926  -308.00645074   -81.50645074 ...  -314.11645074
   -314.11645074  -314.11645074]
 [ 8305.823993     -81.186007     188.813993   ...   -94.176007
    -94.176007     -94.176007  ]
 [ 9934.25797529   -47.68202471   296.08797529 ...   -65.74202471
    -65.74202471   -65.74202471]
 ...
 [23621.9793696   -366.4706304    150.4093696  ...  -378.0206304
   -378.0206304   -378.0206304 ]
 [14663.3477079   -330.1622921    123.0177079  ...  -336.6522921
   -336.6522921   -336.6522921 ]
 [24391.7051857   -594.2248143    -25.6748143  ...  -608.2948143
   -608.2948143   -608.2948143 ]]
Chosen random sample is No.2238
interation (t): 1220
Chosen class is: purpose_other
[[ 1.45696102e+04 -4.12529827e+02 -5.06298274e+01 ... -4.30389827e+02
  -4.30389827e+02 -4.30389827e+02]
 [ 1.26897536e+04 -5.77564129e+01  3.08593587e+02 ... -8.52464129e+01
  -8.52464129e+01 -8.52464129e+01]
 [ 1.12079572e+04 -7.77552789e+02 -3.79042789e+02 ...

interation (t): 1231
Chosen class is: purpose_debt_consolidation
[[ 9.75754454e+03 -2.24255457e+02  1.20074543e+02 ... -2.42455457e+02
  -2.42455457e+02 -2.42455457e+02]
 [ 8.68757199e+03 -3.00138010e+02 -1.22480103e+01 ... -3.12428010e+02
  -3.12428010e+02 -3.12428010e+02]
 [ 6.00870078e+03 -9.76309224e+02 -7.48669224e+02 ... -9.91299224e+02
  -9.91299224e+02 -9.91299224e+02]
 ...
 [ 1.38847565e+04 -5.98783495e+02 -2.59073495e+02 ... -6.15243495e+02
  -6.15243495e+02 -6.15243495e+02]
 [ 2.16684196e+04 -3.20590431e+02  3.88569569e+02 ... -3.31580431e+02
  -3.31580431e+02 -3.31580431e+02]
 [ 1.73706500e+04 -4.19600028e+02  1.42919972e+02 ... -4.29350028e+02
  -4.29350028e+02 -4.29350028e+02]]
Chosen random sample is No.5449
interation (t): 1232
Chosen class is: purpose_other
[[ 1.45666139e+04 -4.15526086e+02 -5.36260860e+01 ... -4.33386086e+02
  -4.33386086e+02 -4.33386086e+02]
 [ 1.26891310e+04 -5.83790022e+01  3.07970998e+02 ... -8.58690022e+01
  -8.58690022e+01 -8.58690022e+01]
 [ 1.

In [33]:
dfSample1400 = dfS
dfSample1400.to_csv('sample1400.csv')